##### Copyright 2019 The TensorFlow Authors.

In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ucshg\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [3]:
level = int(input("Enter the reading level :\n 2 - BASIC\n 1 - MID\n 0 - COMPLEX"))
path_to_file = '../data/shakespeare.txt'#tf.keras.utils.get_file('story.train.2.txt','https://storage.cloud.google.com/cs229data/story.train.2.txt')
if level == 0:
    path_to_file = '../data/shakespeare.txt'#tf.keras.utils.get_file('story.train.2.txt','https://storage.cloud.google.com/cs229data/story.train.2.txt')
elif level == 1:
    path_to_file = '../data/shakespeare.txt'#tf.keras.utils.get_file('story.train.2.txt','https://storage.cloud.google.com/cs229data/story.train.2.txt')
elif level == 2:
    path_to_file = '../data/shakespeare.txt'#tf.keras.utils.get_file('story.train.2.txt','https://storage.cloud.google.com/cs229data/story.train.2.txt')
else:
    print("INVALID LEVEL CODE. GOING WITH THE DEFAULT (LEVEL 0)")

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 121020 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

ACT I
SCENE I. Venice. A street.
Enter ANTONIO, SALARINO, and SALANIO
ANTONIO
In sooth, I know not why I am so sad:
It wearies me; you say it wearies you;
But how I caught it, found it, or came by it,
What stuff 'tis made of, whereof it is bor


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training examples and targets

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(121020,), dtype=int64, numpy=array([12, 14, 31, ..., 60, 53, 59], dtype=int64)>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

A
C
T
 
I



S
C
E


In [17]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'A' b'C' b'T' b' ' b'I' b'\r' b'\n' b'S' b'C' b'E' b'N' b'E' b' ' b'I'
 b'.' b' ' b'V' b'e' b'n' b'i' b'c' b'e' b'.' b' ' b'A' b' ' b's' b't'
 b'r' b'e' b'e' b't' b'.' b'\r' b'\n' b'E' b'n' b't' b'e' b'r' b' ' b'A'
 b'N' b'T' b'O' b'N' b'I' b'O' b',' b' ' b'S' b'A' b'L' b'A' b'R' b'I'
 b'N' b'O' b',' b' ' b'a' b'n' b'd' b' ' b'S' b'A' b'L' b'A' b'N' b'I'
 b'O' b'\r' b'\n' b'A' b'N' b'T' b'O' b'N' b'I' b'O' b'\r' b'\n' b'I' b'n'
 b' ' b's' b'o' b'o' b't' b'h' b',' b' ' b'I' b' ' b'k' b'n' b'o' b'w'
 b' ' b'n' b'o'], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'ACT I\r\nSCENE I. Venice. A street.\r\nEnter ANTONIO, SALARINO, and SALANIO\r\nANTONIO\r\nIn sooth, I know no'
b't why I am so sad:\r\nIt wearies me; you say it wearies you;\r\nBut how I caught it, found it, or came by'
b" it,\r\nWhat stuff 'tis made of, whereof it is born,\r\nI am to learn;\r\nAnd such a want-wit sadness makes"
b' of me,\r\nThat I have much ado to know myself.\r\nSALARINO\r\nYour mind is tossing on the ocean;\r\nThere, w'
b'here your argosies with portly sail,\r\nLike signiors and rich burghers on the flood,\r\nOr, as it were, '


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'ACT I\r\nSCENE I. Venice. A street.\r\nEnter ANTONIO, SALARINO, and SALANIO\r\nANTONIO\r\nIn sooth, I know n'
Target: b'CT I\r\nSCENE I. Venice. A street.\r\nEnter ANTONIO, SALARINO, and SALANIO\r\nANTONIO\r\nIn sooth, I know no'


### Create training batches

In [24]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the model

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([34, 15, 36, 37, 30,  2, 22, 28, 15, 29, 18, 22, 62, 59, 48, 30,  4,
       15, 18, 33, 42, 34, 51, 16, 14, 42, 28, 23, 49, 40, 48, 39, 47, 42,
       48, 34, 56, 52, 32, 18,  9,  4, 64, 28,  1, 30, 27, 25, 28, 23, 51,
        3, 23, 32, 57,  4, 15, 23, 27, 33,  2, 61, 29, 31, 61, 24, 14, 58,
       32, 47, 58, 19,  0, 30, 38,  9, 42, 28, 48, 60, 51, 49, 44, 44,  0,
       11, 46, 17, 45, 26, 37,  6,  5, 15, 64, 25, 34, 11, 61, 53],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ther\r\nLorenzo and his amorous Jessica:\r\nBesides, Antonio certified the duke\r\nThey were not with Bass'

Next Char Predictions:
 b"WDYZS\rKQDRGKwtiS!DGVcWlECcQLjai]hciWqmUG:!yQ\nSPNQLl LUr!DLPV\rvRTvMCsUhsH[UNK]S[:cQiuljee[UNK]?gFfOZ,'DyNW?vn"


## Train the model

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.189276


In [35]:
tf.exp(mean_loss).numpy()

65.97502

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [38]:
EPOCHS = 30
if level == 0:
    EPOCHS = 30
elif level == 1:
    EPOCHS = 150
elif level == 2:
    EPOCHS = 200
else:
    EPOCHS = 30

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/150
18/18 [==============================] - 76s 4s/step - loss: 4.1621
Epoch 2/150
18/18 [==============================] - 72s 4s/step - loss: 3.3218
Epoch 3/150
18/18 [==============================] - 76s 4s/step - loss: 2.8893
Epoch 4/150
18/18 [==============================] - 77s 4s/step - loss: 2.5759
Epoch 5/150
18/18 [==============================] - 77s 4s/step - loss: 2.3992
Epoch 6/150
18/18 [==============================] - 77s 4s/step - loss: 2.3044
Epoch 7/150
18/18 [==============================] - 75s 4s/step - loss: 2.2360
Epoch 8/150
18/18 [==============================] - 68s 4s/step - loss: 2.1751
Epoch 9/150
18/18 [==============================] - 78s 4s/step - loss: 2.1182
Epoch 10/150
18/18 [==============================] - 59s 3s/step - loss: 2.0628
Epoch 11/150
18/18 [==============================] - 71s 4s/step - loss: 2.0038
Epoch 12/150
18/18 [==============================] - 65s 4s/step - loss: 1.9485
Epoch 13/150
18/18 [=================

## Generate text

In [40]:
# if level == 0:
#     model.save('level_0.model')
# elif level == 1:
#     model.save('level_1.model')
# elif level == 2:
#     model.save('level_2.model')
# else:
#     model.save('level_0.model')



In [41]:
# if level == 0:
#     model = tf.keras.models.load_model('level_0.model', custom_objects=None, compile=True)
# elif level == 1:
#     model = tf.keras.models.load_model('level_1.model', custom_objects=None, compile=True)
# elif level == 2:
#     model = tf.keras.models.load_model('level_2.model', custom_objects=None, compile=True)
# else:
#     model = tf.keras.models.load_model('level_0.model', custom_objects=None, compile=True)

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO: fie, fie!
SALARINO
Not in love neither? If I do know your tongue.
LORENZO
Lorenzo, and thy love.
JESSICA
Nay, but ask me, when I thought
With the doctor a rest and lought his own intortured
Dot your dost bear unto the praising of myself;
Therefore no more of it: hear other things.
Lorenzo, I commit into your fair heappien
His tedilound me, I must go with me; peruse this as thou goest:
Fair Jessica shall be my torch-beare r.
Exeunt

SCENE II. Belmont. A room in PORTIA'S house.
Enter NERISSA with a Servitor
NERISSA
Quir, I rear duscenish of cornets. Enter the PRINCE OF MOROCCO and his train; PORTIA, NERISSA, and others attending
part of our hisbees of brothes of sucher,
us the fery draw contentaning
eye our absine a daughter: twenty merchants,
The duke himself, and the magnificoes
That did sevent turn you merry.
ANTONIO
I not do but note a wild and water-right:
I have sent twenty out to seek for you.
GRATIANO
I am glad on't: I desire no more than I had,
Your 

____________________

In [45]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO: There, then, let me provert with him\r\nTo one that would have him help to waste\r\nThan is thy strange apparent cruelty;\r\nAnd where thou now exact'st the penalty,\r\nWhich here appeareth due upon the bond.\r\nSHYLOCK\r\n'Tis very wleng Lord, what have at earing the\r\nmesty taken fren to becamed\r\nTo Christian in oath: so for that I choose that the\r\ngrace of Lord Bassanio and myself meantime\r\nWill live as maids and widows. Come, away!\r\nFor we must measure twenty miles to-day.\r\nExeunt\r\n\r\nSCENE V. The same. Before SHYLOCK'S house.\r\nEnter SHYLOCK and LAUNCELOT\r\nSHYLOCK\r\nWell, thou shalt see, there is something else.\r\nThis bond doth give thee here no jow his mere\r\nCannot in prayith, for if me Tord.\r\nNERISSA\r\nWho, if he break, thou mayst with better face\r\nExact the penalty. He will leave you then thirk--\r\nGRATIANO\r\nA second Daniel, a Daniel, Jew!\r\nNow, infidel, I have you on the hip.\r\nPORTIA\r\nWhy doth the greater ngrownight let 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets


In [47]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO: The man is, notwithstanding,
sufficient. Three thousand ducats; I think thee constites.
BASSANIO
Thy dake thou art time: I pray you, which is the way
to master Jew's?
LAUNCELOT
Turn up on your reish;
I'll have my bond; I will try confusions with him.
GOBBO
Lord, how art thou changed! Mark you are welcome notwithstanding.
BASSANIO
To stay for your leave;
I come by note, to give and to return to their
home and gold and such a night
Did prest thou dest debrew of my tribe,
There are same was a wearthy of thy plinc;
I fair one who shall thinks This ring,
Which when you part from, lose, or give away,
Nor this daughter, come in your bed
Until I see the ring.
NERISSA
Nor I in yours
Till I again the very life
Of my dear friend. What, no merchants most widh the
conseiture true so exceastity of cornets
one upon the prince, then stood as fair
As any comer I have look'd on yet
For paring question of my boy
As to thy friends; for when did friendship take
A breed for bar


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [49]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [50]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [51]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [52]:
model.fit(dataset, epochs=5)

Epoch 1/5
18/18 [==============================] - 78s 4s/step - loss: 4.1781
Epoch 2/5
18/18 [==============================] - 69s 4s/step - loss: 3.2898
Epoch 3/5
18/18 [==============================] - 71s 4s/step - loss: 2.8648
Epoch 4/5
18/18 [==============================] - 74s 4s/step - loss: 2.5535
Epoch 5/5
18/18 [==============================] - 76s 4s/step - loss: 2.3868


Or if you need more control, you can write your own complete custom training loop:

In [54]:
EPOCHS = 3

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.3103

Epoch 1 Loss: 2.2962
Time taken for 1 epoch 63.80 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.2252

Epoch 2 Loss: 2.2265
Time taken for 1 epoch 63.87 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.2035

Epoch 3 Loss: 2.1665
Time taken for 1 epoch 66.33 sec
________________________________________________________________________________


In [58]:
start = time.time()
states = None
print('Level - ',level)
print('\n')
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Level -  1


ROMEO: fie, fie,
I art their follow in a gondear are her toaching
That they shall think we are accomplished
With that we lack. I'll hold thee any wager,
Which pircus ofe hird, than there is between red
Before the curt of Venice.
Music thent? Cherisame. I am well about
endy wo letter; and if my fortune be not crost,
I have a father, you a daughter, lost.
Exit

SCENE VI. The same. A street.
Enter GRATIANO, LORENZO, SALARINO, and Sellocks:
The outwore, choise me ans upright flack,
In cheruse the seme bath rubser fare that supper
be ready at the farthest by five of the clock:
Shylock thy master spoke with me this day,
Nor well, unto the gracen of the twentieth part
Of one poor scruple, nay, i' lies not me out;
And there is such confusion in my wish,
To wish myself much better than the wren.
How many then should cover the truth wing
And would conceive for what I gave the ring
And how unwillingly I left the ring.
Exit

PORTIA
Come on, Nerissa; I have work in han
The pou 

______